# Mysql系统基础
* 系统配置文件
    * 启动时会去读取mysql的配置文件，```mysql --help | grep my.cnf``` 查看配置文件读取顺序，后面的会覆盖前面的
        * 加载顺序：```my.cnf, MYSQL_TCP_PORT, /etc/my.cnf /etc/mysql/my.cnf /usr/local/mysql/etc/my.cnf ~/.my.cnf```
    * 数据库和数据库实例的区别，数据库是静态的二进制数据文件，数据库实例是一些介于操作系统和用户之间的应用程序用来操作数据库文件
* 系统架构：连接池->管理服务工具->SQL接口组件->查询分析器(SQL解析器)->优化器组件(SQL优化器)->缓存组件->存储引擎->物理文件，如下图所示：
    <img src="../images/mysql/mysql架构.png" width="400px">
* 存储引擎对比：
    <img src="../images/mysql/存储引擎对比.png" width="400px">
* 连接方式有TCP/IP，命名管道和共享内存，Unix套接字
* 磁盘列阵介绍：详细情况参考《Mysql技术内幕：InnoDB存储引擎》第316页：<img src="../images/mysql/磁盘阵列对比.png" width="400px">
* 基准测试工具：纯测数据库sysbench, mysql-tpcc, mysqlslap, sql-bench，集成测试ab, http_load；实例参考《高性能mysql》第90页
    * sysbench：测试cpu性能，磁盘io性能，调度程序性能，内存分配以及传输速度，POSIX线程性能，数据库OLTP基准测试
    * msql-tpcc：测试数据库OLTP性能，度量单位tmpC(transaction per minute)，越大性能越好
    * ab：Apache的测试HTTP服务器每秒最多可以处理多少请求
    * http_load：与ab类似
    * JMeter：即可以测数据库，也可以集成测试
* 基准测试性能指标：Percona Tooltik工具集监控收集性能相关数据
    * 吞吐量：单位时间内处理的事务量
    * 响应时间或延迟：一次事务处理的时间
    * 并发性：同时可接受响应请求的数量，并发性提高要关注吞吐量是否下降，响应时间是否变长
    * 可扩展性：增加一倍的系统资源，上述三个指标随之增加一倍（理想情况）
* 排查是单条查询语句的问题还是服务器本身的问题，示例：《高性能mysql》135页
    * 使用```SHOW GLOBAL STATUS;```监控连接线程，正在执行线程数
    * 使用```SHOW PROCESSLIST;```监控线程是否长时间处于不正常的状态
    * 使用慢查询日志
    * 整理上述数据文件画图
    * 设置触发器监控上述数据包括：CPU利用率，磁盘使用率和可用空间，ps的输出采样，内存利用率
* 数据类型选择：
    * 更小的通常更好，但是要确保好范围
    * 简单就好
    * 尽量避免NULL，对innodb影响不大
* 标识列的选择：最好是整型，不要用枚举，避免用字符串，如果使用UUID()应该移除```-```号，最好使用UNHEX()和HEX()转为16进制数存储在BINARY(16)中
* 不好的设计规范：
    * 太多列
    * 太多关联：mysql只支持关联61个表
    * 全能的枚举：过度使用枚举
    * 变相的枚举：使用其他方式来实现枚举
    * 非此发明的NULL：避免出现NULL，强制定义一些没意义的值来替代NULL
* 范式：
    * 范式的好处：<img src='../images/mysql/范式的优点.png' width='450px'>
    * 实际中范式和反范式交叉使用，结合实际需求添加必要的重复数据减少关联提高查询性能
* 慢查询的原因：
    * 是否请求了不需要的数据
        * 查询不需要的记录，比如在SELECT中限制TOP只会返回TOP的结果，实际上mysql会返回所有结果，再根据TOP做过滤
        * 多表关联查询时返回全部的列，SELECT * 坚决不要用
        * 查询所有列，SELECT * 坚决不要用
        * 重复查询同样的数据，可以利用缓存
    * 是否扫描了额外的记录
        * 监控响应时间
        * 监控扫描的行数和返回的行数，比例太高表示扫描效率低
        * 监控扫描的行数和访问类型，访问类型从快到慢：常数引用(constant)，唯一索引查询(ref)，范围扫描(range)，索引扫描(index_key)，全表扫描(all)
        * 应用WHERE条件从好到坏：在索引中使用WHERE条件过滤，存储引擎层；使用索引覆盖直接从索引中找数据，mysql服务器层（Using index）；从数据表中查找数据，mysql服务器层（Using where）
    * 重构查询方法：合理选择复杂查询或者简单查询，切分查询，分解关联查询
* mysql处理查询流程图：<img src='../images/mysql/mysql处理查询流程.png' width='450px'>
* mysql线程状态：Sleep, Query, Locked, Analyzing and statistics, Copy to temp table, Sorting result, Sending data
* mysql优化器能做的事情：
    * 重新定义关联表顺序
    * 将外连接转成内连接
    * 使用等价变化规则
    * 优化MAX(), MIN(), COUNT()函数
    * 预估并转化为常数表达式
    * 覆盖索引扫描
    * 子查询优化
    * 提前终止查询，LIMIT限制
    * 等值传播
    * 列表IN()的比较，会排序使用二分查找
* mysql执行关联查询的逻辑：行嵌套循环关联操作，现在一个表中查找一行数据，然后再嵌套到下一个表中查找，直到嵌套完所有的表找到所有的行数据，最后根据找到的行数据返回需要的列
* 优化特定类型的查询：
    * 优化COUNT()可以用EXPLAIN获得不太精确的值
    * 优化LIMIT，可以记录上条记录的ID，然后从上条记录开始LIMIT
* mysql字符集指定规则：
    * 创建数据库时，将根据服务器上面的character_set_server设定数据库的默认字符集
    * 创建表时，根据数据库的字符集创建默认字符集
    * 创建列时，根据表的字符集创建默认字符集
    * 建立连接时，服务端会认为客户端使用的是character_set_client指定的字符集
    * 通信时，服务端接收到客户端的SQL后，会转为character_set_connection指定的字符集
    * 返回结果时，服务端会按character_set_result指定的字符集返回给客户端
* mysql查询缓存存储的是查询语句的Hash，所以必须完全一样的查询语句才会命中缓存，如果查询语句包含不确定函数如NOW(), CURRENT_DATE()缓存也不会命中
* mysql需要配置的内存结构：每个连接需要的内存，为操作系统保留的内存，为mysql缓存分配内存（查询缓存），为innodb缓存分配内存（索引缓存），线程缓存的内存，表缓存的内存，
* 并发值 = CPU数量 \* 磁盘数量 \* 2
* vmstat和iostat：
<table>
    <tr>
        <td><img src='../images/mysql/vmstat.png' width='450px'></td>
        <td><img src='../images/mysql/vmstat参数1.png' width='450px'></td>
    </tr>
     <tr>
        <td><img src='../images/mysql/vmstat参数2.png' width='450px'></td>
        <td><img src='../images/mysql/iostat.png' width='450px'></td>
    </tr>
    <tr>
        <td><img src='../images/mysql/iostat参数1.png' width='450px'></td>
        <td><img src='../images/mysql/iostat参数2.png' width='450px'></td>
    </tr>
    <tr>
        <td><img src='../images/mysql/iostat参数3.png' width='450px'></td>
    </tr>
</table>
* 数据库扩展，向上扩展添加硬件设备，向外扩展复制、拆分、数据分片
<table>
    <tr>
        <td><img src='../images/mysql/按功能拆分数据库.png' width='450px'></td>
        <td><img src='../images/mysql/数据分片数据库.png' width='450px'></td>
    </tr>
</table>
* 负载均衡的作用：可扩展性，高效性，可用性（平均失效时间MTBF，平均恢复时间MTTR），透明性，一致性
* 负载均衡算法：随机，轮询，最少连接数，最快响应，哈希，权重

****

# InnoDB存储引擎
* innodb线程组成：含1个主线程，1个日志线程，1个锁监控线程，4个读线程，4个写线程
* 主线程循环逻辑：
    * 每一秒钟循环操作内容：刷新重做日志缓存(即使事物没提交)，合并插入缓存(可能发生，当前1秒io次数少于5次时执行)，刷新缓存池中的脏数据页(最多100个，可能发生，当脏数据比超过最大脏数据比例时执行)，切换到后台线程(可能)
    * 每十秒循环操作内容：刷新100个脏页数据(可能发生，过去10秒io次数少于200次)，合并插入缓存(最多5个)，刷新重做日志缓存，删除无用undo页，刷新100个或10个脏页数据，产生检查点
    * 后台循环：删除无用undo页，合并插入缓存，调回主循环线程，跳转到刷新线程(可能发生)
    * 刷新循环
    * 挂起循环
* innodb关键特性：插入缓存，两次写（写内存拷贝数据，写重做日志），自适应hash
* innode存储逻辑，详细情况参考《Mysql技术内幕：InnoDB存储引擎》第115页
    * 内存组成：缓冲池，重做日志缓存池，内存池；按页(16KB)读取文件到缓存池中，按最近最少使用(LRU)算法更新缓存池中的数据，更新数据也是最先发生在缓存池中，然后按一定频率刷新到文件中
    * 缓冲池中有索引页，数据页，undo页，插入缓存，自适应哈希索引，innodb存储的锁信息，数据字典信息
    * innodb存储引擎文件包括表文件和重做日志文件，表空间文件以ibd结尾，表结构文件以frm结尾
    * 存储空间逻辑图：独立表空间文件的数据、索引、插入缓存在表空间中，而其他还是在共享表空间中
        <img src='../images/mysql/存储引擎存储空间.png' width='500px'>
    * 段由数据段，索引段，回滚段组成，数据段为B+树的叶子节点，索引段为B+树的非叶子节点
    * 区由64个连续的页组成，每个页大小16KB，每个区1MB，页是最小的存储单位，页结构如下图
        * 文件头，指定该页在表空间中的具体偏移量，位置以及页类型
        * 页头，记录数据页的状态信息
        * Infimum和Supremum用来分隔数据列
        * User Records与FreeSpace实际记录数据和空闲的数据空间
        * Page Directory记录数据在页中的相对位置
        * 文件结尾，记录和文件头相关的验证信息，保证页数据的完整
        <img src='../images/mysql/存储引擎页结构.png' width='300px'>
    * innodb两种行数据的排列格式，老的行数据格式；新的行数据格式为Compressed和Dynamic，新数据格式完全会采取溢出存储数据，也就是对于varchar数据页只存储前20字节，其他全部放入blob页，Compressed则会对blob，text大数据列采取zlib压缩算法：
        * innodb对于所有varchar总字节数不能超过$2^{16}$字节，varchar(N)，N指的是字符数， 但数据列数据大于8098字节时，会将数据存放到blob页中，而实际的数据页只存放前768字节和指向blob页的指针
        * innodb的char(N)其实也是字符长度，所以本质来说char也是变长的，因为两个字符长度的中文和英文占用的字节数是不一样的
        <img src='../images/mysql/存储引擎行结构1.png' width='500px'>
        <img src='../images/mysql/存储引擎行结构2.png' width='500px'>   
* 预读取：
    * 顺序读，从B+树的叶子节点顺序读取数据，逻辑上是顺序的，物理上是离散的，但是大部分还是顺序的
    * 随机读，从辅助索引/非聚合索引叶子节点读取key，然后到聚合索引叶子节点读取数据，逻辑和物理都是随机的
    * 随机预读取，当缓存中存在一个区的13个页并且在LRU列表的前端，则会将该区中的所有页都读入缓存，已取消
    * 顺序预读取，当一个区中的24个页都被顺序访问过，则会下一个区的所有页
* innodb锁
    * 锁级别是行数据，一致性的非锁定读
    * 锁的类型：
        * 共享锁（S Lock），允许事务读一行数据
        * 排他锁（X Lock），允许事物删除或更新一行数据
        * 意向共享锁，允许获取表中多行数据的共享锁
        * 意向排他锁，允许获取表中多行数据的排他锁
    * INFORMATION_SCHEMA中的INNODB_TRX,INNODB_LOCKS,INNODB_LOCK_WAITS监控当前事物下的锁状态，详细情况参考《Mysql技术内幕：InnoDB存储引擎》第219页
    * 一致性的非锁定读，通过多版本行数据实现，多版本并发控制MVCC
    * 事务隔离级别：
        * READ UNCOMMITTED
        * READ COMMITTED和REPEATABLE READ（默认隔离级别），使用一致性的非锁定读
        * READ COMMITTED总是读取被锁定行的最新快照数据，使用Record Lock算法
        * REPEATABLE READ总是读取事务开始时的行版本数据，使用Next-Key Lock算法
        * SERIALIZABLE，每次读都会添加共享锁，主要用于分布式数据库中
    * 锁的算法：
        * Record Lock，单个行记录上的锁
        * Gap Lock，间隙锁，锁定一个范围，但不包括记录本身
        * Next-Key Lock：Record Lock+Gap Lock，锁定一个范围，并包括记录本身
    * 锁产生的问题：
        * 丢失更新，一条记录用户A读取，用户B读取，用户A修改，用户B修改，则用户A的更新丢失了，需要加排他锁，将并行操作强制为窜性操作
        * 脏读分为脏页读取和脏数据读取，违反了隔离性，解决方案就是设置事务隔离级别在READ COMMITTED之上
            * 脏页是指缓存区中的数据被更新了但是还没刷新到磁盘中，属于正常现象
            * 脏数据是指缓存中的数据被更新了但是没有提交，也就是说一个事务读到了另一个事务还在修改中的数据
        * 不可重复读（幻象现象），在同一事务内多次读同一数据，出现数据不一致，也就是事务第一次读该数据和第二次读该数据时，有另外一个数据对其进行了修改（并且已经提交），导致数据不一致，违反了一致性，解决方案就是设置事务隔离级别在REPEATABLE READ之上
        * 阻塞
        * 死锁
* 编译和调试（深入源码）：详细情况参考《Mysql技术内幕：InnoDB存储引擎》第337页
* innodb + Sphinx：实现全文检索

****

# 索引
* 聚集索引和非聚集索引（辅助索引）：
    * 聚集索引的主键最好用自增型，禁止使用UUID，因为插入时越有顺序，聚集索引效率越高，空间利用率越高
    * 聚集索引表示表中存储的数据按照索引的顺序存储，检索效率比非聚集索引高，但对数据更新影响较大
    * 聚集索引按每张表的主键构建一颗B+树，叶子节点存放所有的数据记录也就是数据页，每张表只能有一个聚集索引
    * 非聚集索引表示数据存储在一个地方，索引存储在另一个地方，索引带有指针指向数据的存储位置，非聚集索引检索效率比聚集索引低，但对数据更新影响较小
    * 非聚集索引的叶子节点不存储实际的数据记录，而是存储一个指向主键索引/聚集索引的主键，所以非聚集索引查询的时候会先找到叶子节点的主键，然后通过该主键到聚集索引中查询具体的数据页，每张表可以有多个非聚集索引
    * 聚集索引的更新方式是创建临时表，并把原表导入进来；非聚集索引更新方式是加S锁，然后允许读操作，删除操作就是直接标记该索引空间可用
* 低选择性的列不需要建立B+树索引，高选择性的列可以建立B+树索引
    * 低选择性意味着列值可选范围很小，数据多，如性别列
    * 高选择性意味着列值可选范围很大，数据少，几乎没重复性
    * 但如果是在高选择性列上面查询很大范围的数据时（全部数据的20%），mysql就不会使用B+树索引了（即使创建了索引），会进行全表扫描
* 联合索引
    * 对多个列建立索引
    * 也是B+树，但是节点中的key是一个多元组，元组值顺序按索引定义时的列顺序
    * 假如联合索引是(a, b)，则```WHERE a=XXX AND b=XXX; WHERE a=XXX;```都会使用索引，而```WHERE b=XXX;```则不会
    * 利用联合索引对第二个key进行排序，可以避免一次额外的排序操作
* Hash索引
    * Hash索引只包含哈希值和行指针
    * Hash索引不是按索引值顺序存储的，所以是无序的
    * Hash索引不支持部分索引列查找，因为Hash值计算的是所有索引列
    * Hash索引只支持等值比较查找，如```=, IN(), <=>;```，不支持范围查找
    * Hash所有查询速度很快，一般是O(1)，除非有很多冲突，如果冲突多了，索引的维护代价很高
    * 存储无规则的数据时，如URL，对该无规则数据添加一列Hash值```crc32(), MD5(), SHA1()```，并将Hash值加入到B+Tree索引中，创建触发器实现
* 空间数据索引(R-Tree)
* 全文索引：可以使用前缀索引，找出列的最大不重复长度，用该值建立Hash索引
* 索引优化器解析路径：
    * <img src='../images/mysql/索引解析路径.png' width='400px'>
    * 匹配列指在索引树中参与匹配的关键字，也就是说会通过索引查找的列
    * 过滤因子 = (结果集行数) / (表的总记录数)
    * 平均过滤因子 = 1 / (不重复列值的个数)
    * 选择率 = 100 * (某个键值对应的行数) / 表的总记录数
* 三星索引：索引列排序时，选择性高的放前面
    * 索引列是相邻的，或者相距很近，索引列定义的顺序和表创建时列顺序一致
    * 索引列的顺序与查询语句中出现的顺序一致，WHERE中谓词的顺序和索引列定义的顺序一致
    * 索引列中的列包含了查询语句中的所有列，SELECT中的列全部出现在索引列中
    * 发生冲突时，如果要保存索引自动排序功能，保留第二颗星；如果要尽量缩小索引行数据，保留第一颗星
* 索引的匹配规则以及限制
    * 全值匹配：和索引中的列进行全部匹配
    * 匹配最左前缀：只使用索引列的最左边的一些列
    * 匹配列前缀：只使用索引列的第一列
    * 匹配范围值：只使用索引列第一列的范围值
    * 精确匹配某一列并范围匹配另一列：只使用索引列第一列精确匹配和第二列范围匹配
    * 只访问索引的查询（覆盖索引）：索引列包含所有需要查询的列
    * 必须按照索引的最左列开始查找，也就是谓词的顺序要和索引列一直
    * 不能跳过索引中的列
    * 索引列中某个列使用了范围查找，则该列后面的索引列都不能使用索引了
* Innodb和MyISAM索引组织数据的区别：<img src='../images/mysql/存储引擎数据组织区别.png' width='450px'>
* 存储引擎索引不支持LIKE通配符操作，最多支持匹配列前缀的LIKE操作
* 使用索引排序：索引列的顺序和ORDER BY字句的顺序一致，并且排序方向一致，会使用索引排序；有多张表关联查询时ORDER BY中的列必须全部为第一张表中的列，会使用索引排序
* 要删除重复索引和从不使用的索引，优化冗余索引（可删可改）

****

# 事务
* 事务四大特性ACID：
    * 原子性：一个事务是一个最小的不可分割的操作单元，要么全部成功要么全部失败
    * 隔离性：一个事务在最终提交前，对其他事务是不可见的
    * 一致性：数据库总是从一个一致性状态转变为另一个一致性状态
    * 持久性：一旦事务被提交，其修改永久性的保存到数据库中
* 隔离性有锁实现，原子性、一致性、持久性通过redo和undo实现
    * redo：事务日志通过innodb重做日志文件和innodb日志缓存实现，事务开始记录一个事务日志编号LSN到日志缓存，事务执行时将执行日志全部写入日志缓存，事务提交刷新缓存的数据到重做日志文件中，这种方式叫做预写日志方式
    * undo：存放在undo段中，记录事务回滚的操作
* 事务控制语句：```START TRANSACTION | BEGIN, COMMIT, ROLLBACK, SAVEPOINT identifier, RELEASE SAVEPOINT identifier, ROLLBACK TO [SAVEPOINT] identifier, SET [GLOBAL | SESSION] TRANSACTION ISOLATION LEVEL```
* COMMIT, COMMIT WORK和RELEASE, RELEASE WORK几乎是一样的，只有设置completion_type大于0时有略微差别，completion_type取值范围$[0, 1, 2]$
* 隐式提交事务的SQL类型：DDL语句、隐式修改mysql架构的操作、管理语句
* 性能指标：每秒请求数QPS，每秒事务处理能力TPS（$\frac{com\_commit + com\_rollback}{time}$，且必须是显示提交的才计算）
* 分布式事务：
    * <img src='../images/mysql/分布式事务组成.png' width='400px'>
    * 两段式提交：
        * 第一阶段，所有事务参与者（上图的资源管理器）告诉事务管理器准备就绪
        * 第二阶段，事务管理器通知事务参与者COMMIT或ROLLBACK
    * Java中的JTA(Java Transaction API)相当于上图的应用程序，负责协调事务管理器和事务参与者
* 不好的事务使用习惯
    * 在循环中提交事务，首先非常耗性能（提交一次写一次重做日志），其次mysql默认是开启了自动提交的
    * 使用自动提交，```SET autocommit=0```关闭自动提交，或者使用```START TRANSACTION | BEGIN```显示开启一个事务，这时mysql会自动关闭自动提交，等事务结束，mysql会自动打开自动提交
    * 使用自动回滚

****

# 数据库恢复和备份
* 备份类别：
    * 备份方法：
        * 热备份：在数据库运行时直接备份，对正在运行的数据库没有影响，工具：ibbackup, xtrabackup
        * 冷备份：在数据库停止运行时备份，一般就是拷贝数据库文件即可
        * 温备份：在数据库运行时备份，备份时会对数据库加全局锁，对正在运行的数据库造成影响
    * 备份后文件的内容：
        * 逻辑备份：记录的是SQL语句，用户数据库升级、迁移等，非常耗时，工具：mysqldump
        * 裸文件备份：其实就是数据库文件
    * 备份内容：
        * 完全备份：对数据库做一个全面的备份
        * 增量备份：在上次备份的基础上备份
        * 日志备份：利用binlog实现备份
* 数据库复制流程图：<img src='../images/mysql/数据库复制流程.png' width='300px'>
* bin-log有基于语句STATEMENT和基于行ROW的两种方法，binlog-format设置类别

****

# 约束
* 数据完整性包括实体完整性（主键），域完整性（数据的特殊约束），参照完整性（外键），传统的check约束由以下两种方式实现：
    * 通过ENUM实现，并设置sql_mode为STRICT_TRANS_TABLES```CREATE TABLE test(sex ENUM('male', 'female'));```
    * 通过触发器实现

****

# 视图
* 视图可更新，但是需要加上```WITH CHECK OPTION```做检查
* 不支持物化视图，只有虚拟视图，所以如果要实现物化视图需要自己单独定义一张表，并以MV结尾表意
* 视图的两种实现算法对比：<img src='../images/mysql/视图算法对比.png' width='450px'>

****

# 存储代码
* 存储代码包括：存储过程，存储函数，触发器，定时任务
* 存储过程示列：<img src='../images/mysql/存储过程示例代码.png' width='400px'>
* 存储代码优缺点：
    * 在服务器内部执行，离数据最近，可以节省网络带宽和延迟
    * SQL代码可以重用
    * 简化代码的维护和更新
    * 可以提高安全性
    * 服务器可以缓存存储过程执行计划，对重复调用性能比较好
    * 在服务器上部署，所以备份，维护都在服务器上完成
    * 可以使应用开发人员和数据库开发人员更好的分工
    * mysql没有好的调试工具可以对sql函数代码调试
    * 和应用程序代码对比，性能略差
    * 会给整个系统的部署带来额外的开销
    * 会给数据库服务器增加额外的压力
    * mysql没有选项可以控制存储过程的资源消耗，很容易拖垮服务器
    * 存储过程实现时很多方法没有，没有应用程序代码实现逻辑简单
    * 调试存储过程非常困难，因为不会报错到具体的哪行SQL
    * 和基于STATEMENT的二进制日志兼容性不是很好
* 触发器：
    * 触发器对每个表的每个事件只支持一个触发器，也就是不能在AFTER INSERT上面建立两个触发器
    * mysql只支持基于行的触发器，只能针对一行数据触发触发器
    * 触发器会掩盖服务器背后的工作
    * 触发器的问题很难排查，尤其是性能问题发生在触发器中时
    * 触发器可能导致死锁和锁等待，如果触发器失败，原来的SQL也会失败，很难发现问题

****

# 表分区
* mysql支持的分区表类型：（如果存在主键或者唯一索引分区列必须是唯一索引的一部分，否则不需要）
    * RANGE分区按列值的连续区间分区
    * LIST分区按列值的离散区间分区
    * HASH分区根据用户自定义的HASH值分区
    * KEY分区根据mysql自定义的HASH函数分区
    * COLUNMS分区（支持非整形数据分区，其他四种只支持整形数据分区）
    * RANGE和LIST分区允许再使用HASH和KEY做进一步子分区，且每个自分区数量必须相同，并且一旦创立自分区，所有分区都要建立
    * RANGE分区处理NULL值会将其放入最左边的分区
    * LIST分区处理NULL则需要在LIST中间明确指定NULL存放在哪个分区中
    * HASH和KEY分区则会将NULL转为0进行存储
* 需不需要创建分区表看数据量的大小，如果分区能够降低B+树的层次，则可以尝试，否则没必要，因此大部分数据例如1000万条也最多3层B+树，因此查询只需要2-3次IO，而分成10区后则需要20次IO
* 分区的限制：
    * mysql只支持1024个分区表，并且分区表不能包含外键
    * NULL值会使分区过滤无效，因为NULL默认会插入到分区的第一个表中，查询的时候就会扫描两个分区，如果第一个表太大就失去了分区的意义
    * 分区列和索引列不匹配，这样就会导致每个分区表都会有一个同样的索引，所以查询走索引的时候还是会扫描所有分区
    * 选择分区的成本会很高，尤其是插入数据的成本，因为要去查找插入数据属于哪个分区，100个分区表是可以接受的
    * 打开并锁住所有底层表的成本会很高，查询分区表时，会在底层把所有表都锁住
    * 维护分区表的成本很高，这里说的是修稿分区表的时候
    * 所有分区必须使用相同的存储引擎
    * 分区函数可以使用的函数和表达式有限制
    * 某些存储引擎不支持分区
* 有分区表做查询的时候，记得在谓词中加上分区列，这样可以过滤分区数据

****

# 配置系统变量
* datadir：指定数据库文件路径 
* innodb_read_io_threads，innodb_write_io_threads：可以改变默认读写线程数量
* innodb_buffer_pool_size，innodb_log_pool_size：可以改变缓存池和重做日志缓存池
* innodb_flush_log_at_trx_commit：设置重放日志缓存刷新规则，0表示不根据commit提交，只根据时间刷新，1表示commit时刷新，2表示异步刷新也就是说commit的时候不一定会刷新只会有这么一个动作
* innodb_file_per_table：指定表空间是否由独立的文件构成
* innodb_read_ahead_threshold：指定该区最小多少个页被访问过才会触发顺序预读取
* innodb_adaptive_hash_index：设定自适应hash索引
* innodb_autoinc_lock_mode：设置插入自动增长主键数据时的锁类型，0表示表级锁，1多种组合锁级别（默认值），2互斥量锁（可能插入数值会不连续）
* innodb_lock_wait_timeout：设置阻塞等待时间，模式50s
* innodb_rollback_on_timeout：设置阻塞等待时间超时时回滚操作，默认关闭
* ```SHOW ENGINE innodb STATUS;```中的```log sequence number, log flushed up to, last checkpoint at```查询日志缓存和重做日志之间的差值
* innodb_use_sys_stats_table：打开表时将表的统计信息持久化
* autocommit：设置是否自动提交，默认为1开启自动提交（隐式提交），0关闭自动提交
* transaction-isolation：设置事务隔离级别
* @@tx_isolation：查询事务隔离级别使用的变量
* innodb_locks_unsafe_for_binlog：设置binlog在STATEMENT模式下使用READ COMMITTED事务隔离级别（建议binlog日志模式改为ROW）
* binlog日志配置参数：<img src='../images/mysql/binlog日志配置参数.png' width='150px'>
* innodb_support_xa：查看是否开启分布式事务
* 缓存池命中率计算：<img src='../images/mysql/缓存池命中率计算公式.png' width='350px'>
* character_set_database：设置数据库默认字符编码
* query_cache_min_res_unit：缓存分配内存块时的最小单位
* query_cache_type：是否打开查询缓存
* query_cache_size：查询缓存使用的总内存大小
* query_cache_limit：缓存最大的结果集，查询结果超过这个值就不会被缓存了
* query_cache_wlock_invalidate：判断当数据被锁后，是否还能从缓存中读出数据，默认是关闭的
* 基础配置选项：<img src='../images/mysql/mysql基础配置选项.png' width='450px'>
* mysqladmin extended-status -ri60：监控服务器状态数据